# Método de puntos interiores
### Fátima Ginebra

We do not know if this is the possible best version but here goes nothing...

In [7]:
import numpy as np
import copy as copy

In [8]:
def paso_intpoint(mu, wmu):    
    alphas = np.ones((len(wmu)))
    for i in range(len(wmu)):
        if wmu[i] < 0:
            alphas[i] = -mu[i]/wmu[i]
    alpha = min(alphas)
    alfa = min(alpha, 1.0)        
    return alfa

In [9]:
def intpoint(Q, A, F, c, b, d):
    k = 0
    n = Q.shape[0]
    m = A.shape[0]
    p = F.shape[0]
    tol = 10**(-6)
    kmax = 30
    tau = 0.5

    print("El rango de A es", np.linalg.matrix_rank(A))

    AT = A.T
    FT = F.T

    # Initial values
    x = np.ones(n)
    lamda = np.zeros(m)
    mu = np.ones(p)
    z = np.ones(p)

    e = np.ones(p)

    # Initial residuals
    v1 = Q @ x + AT @ lamda - FT @ mu + c
    v2 = A @ x - b
    v3 = -F @ x + d + z
    v4 = np.multiply(mu, z)  # Use element-wise product for complementarity condition

    ld = np.concatenate((v1, v2, v3, v4), 0)
    norma_cnpo = np.linalg.norm(ld)

    while norma_cnpo > tol and k < kmax:
        # Update diagonal matrices Z and U inside the loop
        Z = np.diag(z)
        U = np.diag(mu)

        ### KKT Matrix
        row1 = np.hstack((Q, AT, -FT, np.zeros((n, p))))
        row2 = np.hstack((A, np.zeros((m, m + p + p))))
        row3 = np.hstack((-F, np.zeros((p, m + p)), np.identity(p)))
        row4 = np.hstack((np.zeros((p, n + m)), Z, U))

        M = np.vstack((row1, row2, row3, row4))

        # Perturb the complementarity condition (v4)
        v4_pert = np.multiply(mu, z) - tau * e
        ld_pert = np.concatenate((v1, v2, v3, v4_pert), 0)

        # Solve the linear system with the perturbed residual
        w_vector = np.linalg.solve(M, -ld_pert)

        wx = w_vector[0:n]
        wlamda = w_vector[n:n + m]
        wmu = w_vector[n + m:n + m + p]
        wz = w_vector[n + m + p:]

        ### Step size
        alpha_mu = paso_intpoint(mu, wmu)
        alpha_z = paso_intpoint(z, wz)
        alpha = 0.95 * min(alpha_mu, alpha_z)

        # Update variables
        x += alpha * wx
        mu += alpha * wmu
        lamda += alpha * wlamda
        z += alpha * wz

        # Update tau and residuals
        tau = np.dot(mu, z) / (2 * p)
        k += 1

        # Recalculate residuals
        v1 = Q @ x + AT @ lamda - FT @ mu + c
        v2 = A @ x - b
        v3 = -F @ x + d + z
        v4 = np.multiply(mu, z)  # Element-wise product

        ld = np.concatenate((v1, v2, v3, v4), 0)
        norma_cnpo = np.linalg.norm(ld)

        print("iter=", k, "|", "||cnpo||=", norma_cnpo)
        print("Condition number of M:", np.linalg.cond(M))
        print("Equality constraint (A x - b):", np.dot(A, x) - b)
        print("Inequality constraint (F x - d):", np.dot(F, x) - d)

        if norma_cnpo <= tol or k == kmax:
            return x, lamda, mu, z, k

    return x, lamda, mu, z, k


Ejemplo 1

In [10]:
# Define the quadratic matrix Q
Q = np.array([[3, -1, 0],
              [-1, 2, -1],
              [0, -1, 1]])

# Define the linear term vector c
c = np.array([-1, -1, -1])

# Define the equality constraint matrix A and vector b
A = np.array([[1, 2, 1]])  # Coefficients of x1 + 2x2 + x3
b = np.array([4])  # The right-hand side of the equality constraint

# Define an inequality constraint: x1 >= -10
F = np.array([[1, 0, 0]])  # Inequality: x1 >= -10
d = np.array([-10])

# Call the function
x,lamda,mu,z,k = intpoint(Q, A, F, c, b, d)

# Print the results
print(f"Solution for x: {x}")
print(f"Multipliers (y): {lamda}")
print(f"Number of iterations: {k}")

El rango de A es 1
iter= 1 | ||cnpo||= 8.907173328720036
Condition number of M: 5.110404958344173
Equality constraint (A x - b): [0.]
Inequality constraint (F x - d): [10.70986486]
iter= 2 | ||cnpo||= 7.222233838454953
Condition number of M: 4.246236213694781
Equality constraint (A x - b): [0.]
Inequality constraint (F x - d): [10.65960715]
iter= 3 | ||cnpo||= 4.469420258520553
Condition number of M: 4.452327783585633
Equality constraint (A x - b): [0.]
Inequality constraint (F x - d): [10.59837662]
iter= 4 | ||cnpo||= 0.9339489528673716
Condition number of M: 6.757324982116456
Equality constraint (A x - b): [0.]
Inequality constraint (F x - d): [10.52055168]
iter= 5 | ||cnpo||= 0.046697456216911765
Condition number of M: 10.369872833622843
Equality constraint (A x - b): [-4.4408921e-16]
Inequality constraint (F x - d): [10.50102825]
iter= 6 | ||cnpo||= 0.0023349198116000194
Condition number of M: 11.285166414621816
Equality constraint (A x - b): [0.]
Inequality constraint (F x - d): [

Ejemplo 2

In [11]:
Q = np.array([[1, 1, 1],
              [1, 2, 3],
              [1, 3, 4]])

A = np.array([[1, 1, 1]])  # Equality constraint
b = np.array([1])

F = np.array([[1, 0, 0],  # Inequality constraints: x_i >= -100
              [0, 1, 0],
              [0, 0, 1]])

d = np.array([-100, -100, -100])

c = np.array([1, 0, 1])  # Linear term

# Call the intpoint function
x, lamda, mu, z, k = intpoint(Q, A, F, c, b, d)

# Output the results
print(f"Solution for x: {x}")
print(f"Lagrange multipliers (lambda): {lamda}")
print(f"Number of iterations: {k}")


El rango de A es 1
iter= 1 | ||cnpo||= 171.8620627828236
Condition number of M: 21.220041234858023
Equality constraint (A x - b): [1.98112583]
Inequality constraint (F x - d): [100.99370861 101.00157285 100.98584437]
iter= 2 | ||cnpo||= 168.8002876698927
Condition number of M: 60.44775215996477
Equality constraint (A x - b): [1.94583269]
Inequality constraint (F x - d): [101.03119842 100.87955914 101.03507513]
iter= 3 | ||cnpo||= 162.8158739114611
Condition number of M: 48.851370050601695
Equality constraint (A x - b): [1.87684784]
Inequality constraint (F x - d): [101.0617458  100.75318829 101.06191375]
iter= 4 | ||cnpo||= 151.50999308053304
Condition number of M: 50.71834803451829
Equality constraint (A x - b): [1.74652014]
Inequality constraint (F x - d): [101.12674587 100.49302247 101.12675179]
iter= 5 | ||cnpo||= 130.83827632039834
Condition number of M: 92.05175789779639
Equality constraint (A x - b): [1.50822847]
Inequality constraint (F x - d): [101.24588582 100.01645676 101.24

Ejemplo 3

In [12]:
import numpy as np

# Define the problem data
Q = np.array([[2, 0],
              [0, 2]])

A = np.array([[1, 1]])  # Equality constraint x1 + x2 = 1
b = np.array([1])

F = np.array([[1, 0],  # Inequality constraints: x1 >= -10, x2 >= -10
              [0, 1]])

d = np.array([-10, -10])

c = np.array([-4, -6])  # Linear term

# Call the intpoint function
x, lamda, mu, z, k = intpoint(Q, A, F, c, b, d)

# Output the results
print(f"Solution for x: {x}")
print(f"Lagrange multipliers (lambda): {lamda}")
print(f"Number of iterations: {k}")


El rango de A es 1
iter= 1 | ||cnpo||= 13.922201505724084
Condition number of M: 8.032954502104626
Equality constraint (A x - b): [0.90806452]
Inequality constraint (F x - d): [10.9233871  10.98467742]
iter= 2 | ||cnpo||= 11.289781041885409
Condition number of M: 5.093129730492503
Equality constraint (A x - b): [0.73646284]
Inequality constraint (F x - d): [10.73721481 10.99924803]
iter= 3 | ||cnpo||= 7.150137607160681
Condition number of M: 6.203606952839207
Equality constraint (A x - b): [0.4664235]
Inequality constraint (F x - d): [10.46638635 11.00003715]
iter= 4 | ||cnpo||= 2.069340469174307
Condition number of M: 10.298670090125942
Equality constraint (A x - b): [0.13498888]
Inequality constraint (F x - d): [10.13500097 10.99998791]
iter= 5 | ||cnpo||= 0.10346734252608582
Condition number of M: 15.543219642717276
Equality constraint (A x - b): [0.00674944]
Inequality constraint (F x - d): [10.00674896 11.00000048]
iter= 6 | ||cnpo||= 0.0051750999841437065
Condition number of M: 1

In [ ]:
np.random.seed(42)
n = 8
m = 4
p = 10
Q = np.random.rand(n, n)
A = np.random.randn(m, n)
b = np.ones(m)
F = np.random.rand(p, n)
d = np.random.rand(p)
c = np.ones(n)

intpoint(Q, A, F,c,b,d)

### Python functions to remember

In [ ]:
print(np.tril([[1,2,3],[4,5,6],[7,8,9],[10,11,12]]))
print(np.tril([[1,2,3],[4,5,6],[7,8,9],[10,11,12]],-1))

In [ ]:
k = 0
n = 8
m = 4
p = 10

# Valores del problema

Q = np.random.rand(n, n)
Q = np.tril(Q)
Q = np.dot(Q, Q.T)

A = np.random.randn(m, n)
AT = A.T
#rangoA = np.linalg.matrix_rank(A)
b = np.ones(m)

F = np.random.rand(p, n)
FT = F.T
d = np.random.rand(p)
c = np.ones(n)

#Valores iniciales (recordar que es un problema iterativo), todos se van a ir actualizando

x = np.ones(n)
lamda = np.ones(m) #esto es lamda

mu = np.ones(p)
z = np.ones(p)

Z = np.diag(z)
U = np.diag(mu)
e = np.ones(p)

#### Vamos a generar todos los ingredientes para formar nuestra matriz F (sin derivar)
# ------- Generamos todos los renglones por separado y concatenamos

#Lado derecho
tau = 1/2
# v1 = np.dot(Q,x) + np.dot(A.T,lamda) - np.dot(F.T,mu) + c
v1 = Q@x + AT@lamda-FT@mu + c
v2 = A@x - b
#v2 = np.dot(A,x) - b
v3 = -F@x + d + z
#v3 = -np.dot(F,x) + d + z
v4 = U@Z@e - tau*e
#v42 = np.multiply(mu,z) - tau*np.ones(p) Así lo hizo el profe en el lab, es lo mismo, ya lo comprobé
ld = np.concatenate((v1,v2,v3,v4),0)
#ld = np.vstack((v1,v2,v3,v4))

### MATRIZ DE DERIVADAS

row1 = np.hstack((Q,AT,-FT,np.zeros((n,p))))
row2 = np.hstack((A,np.zeros((m,m+p+p))))
row3 = np.hstack((-F, np.zeros((p, m + p)), np.identity(p)))
row4 = np.hstack((np.zeros((p,n+m)),Z,U))

M = np.vstack((row1,row2,row3,row4))

# Solución del sistema lineal

w_vector = np.linalg.solve(M,-ld)
w_vector

wx = w_vector[0:n]
wlamda = w_vector[n:n+m]
wmu = w_vector[n+m:n+m+p]
wz = w_vector[n+m+p:w_vector.shape[0]]

### Ahora va el recorte de paso

## Escogemos alpha_mu y alpha_z.

def paso_intpoint(mu, wmu):    
    alphas = np.ones((len(wmu)))
    for i in range(len(wmu)):
        if wmu[i] < 0:
            alphas[i] = -mu[i]/wmu[i]
    alpha = min(alphas)
    return alpha

alpha_mu = paso_intpoint(mu,wmu)
alpha_z = paso_intpoint(z,wz)
alpha = 0.95*min(alpha_mu,alpha_z)

x += alpha*wx
mu += alpha*wmu
lamda += alpha*wlamda
z += alpha*wz

tau=tau/2
k = k+1

M